In [ ]:
import os
os.chdir('/Users/huhu/Desktop/Data Science for Design/Assignment 3 Project + Report 1000 + Video 2mins 12.7')
os.listdir()
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
import bokeh
import itertools
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.plotting import figure, show
from ipywidgets import interact, interact_manual
from bokeh.io import  show, output_notebook
from bokeh.models import ColumnDataSource
bokeh.io.reset_output()
bokeh.io.output_notebook()

Data Description
After reading in the data from PA-X Data, we can describe our data. These data are selected, since the challenge in this individual assignment is to visualise trends in violent events alongside peace agreement signings with the content of Date, Country, Natural Resources, and Environment. The links between these data and the real world can be seen in this way, if the value for NatRes or LaEn is 1, the peace agreement involves natural resources or environment factors. If the value for NatRes or LaEn is 0, natural resources or environment factors are not contained in the peace agreement.

In [ ]:
pec_df = pd.read_excel('peace agreement.xlsx', index_col='Dat', parse_dates=['Dat']) # read data


In [ ]:
pec_df = pec_df[['Con','NatRes', 'LaEn']]


In [ ]:
pec_df.dtypes # data types

In [ ]:
pec_df.shape # dimension

In [ ]:
pec_df.size # data size

In [1]:
pec_df.info() # data information

NameError: name 'pec_df' is not defined

In [ ]:
np.max(pec_df.index),np.min(pec_df.index) # time span

In [ ]:
pec_df.describe() # summary statistics

# Data Exploration
First, we select those records including NatRes/Enviroment. To initialize our analysis, we select top 10 Countries/Regions according to the Number of agreements signed.

In [1]:
"""
Select Top 10 Countries/Regions according to the Number of agreements signed
"""
countries = pec_df[((pec_df.NatRes==1) | (pec_df.LaEn == 1))].Con.values
countries = [i.split("/") for i in countries]

countries = list(itertools.chain.from_iterable(countries))
countries = [i.split("-") for i in countries]
countries = list(itertools.chain.from_iterable(countries))
countries = [i.replace('(', '').replace(')', '') for i in countries]

name, counts = np.unique(countries, return_counts=True)
cty_df = pd.DataFrame([name, counts], index=['countries', 'counts']).T
most_agreements_cty = cty_df.sort_values('counts', ascending= False).head(10)
most_agreements_cty.set_index('countries').plot(kind = 'bar',figsize = (16,8),rot =35)
plt.title('Top 10 Regions with Natural Resource/Enviroment Relevant Agreement')
plt.ylabel("Number")
plt.show()

NameError: name 'pec_df' is not defined

then, in order to see how they are distributed in time, we depict the develoment trends of such agreements' number over the time. Also, we observe the trends of general peace agreements as comparision. From our result, a multimodal distribution is shown as follows:

In [ ]:
"""
Developing trends of the general peace of agreements.
"""
nat_df = pec_df[((pec_df.NatRes==1) | (pec_df.LaEn == 1))]
nat_df['Month'] = pd.Series(nat_df.index).dt.to_period('M').values

fig = plt.figure()
ax1 = fig.add_subplot(111)
p1 = nat_df.groupby('Month')[['Con']].count().pct_change()
p1.columns =['change']
l1 = p1.plot(figsize = (16,8), ax = ax1)
plt.xlabel('time')
plt.ylabel('Change')
# plt.legend(loc = 'topleft')
ax2 = ax1.twinx()
p2 = nat_df.groupby('Month')[['Con']].count()
p2.columns =['counts']
l2 = p2.plot(figsize = (16,8), ax = ax2,color ='blue')
plt.ylabel('Number')
plt.title("Trend of Resource/Enviroment Agreements' Number(change and counts)")
plt.show()

Next, we look at the data tracking violent events across the world. Actually, in ACLED data set, there exists a variable sub_event_type, and we can select peace agreement recording from it. Then we repeat previous plots and a multimodal distribution is shown as well. Compared with the previous results, we find that:

Most of top ten countries in term of NatRes/LaEn also can be found here.
The trends however are different.

In [ ]:
cflct_df = pd.read_csv('conflict.csv',parse_dates=['event_date'])
agreement_df =cflct_df.loc[cflct_df.sub_event_type=='Agreement']
peace_id = [i.lower().find('peac') >=0 for i in agreement_df.notes.values]
peace_event_df = agreement_df.loc[peace_id]
name, counts = np.unique(peace_event_df.country, return_counts=True)
cty_df = pd.DataFrame([name, counts], index=['countries', 'counts']).T
most_event_cty = cty_df.sort_values('counts', ascending= False).head(10)
most_event_cty.set_index('countries').plot(kind = 'bar',figsize = (16,8),rot =75)
plt.title('Top 10 Regions with Most Recorded Event')
plt.ylabel("Number")
plt.show()

In [ ]:
cflct_df['Month'] = pd.Series(cflct_df.event_date).dt.to_period('M').values

fig = plt.figure()
ax1 = fig.add_subplot(111)
p1 = cflct_df.groupby('Month')[['event_id_cnty']].count().pct_change()
p1.columns =['change']
l1 = p1.plot(figsize = (16,8), ax = ax1)
plt.xlabel('time')
plt.ylabel('Change')
# plt.legend(loc = 'topleft')
ax2 = ax1.twinx()
p2 = cflct_df.groupby('Month')[['event_id_cnty']].count()
p2.columns =['counts']

l2 = p2.plot(figsize = (16,8), ax = ax2,color ='blue')
plt.ylabel('Number')
plt.title("Trend of Peace Agreements' Number(change and counts)")
plt.show()

Comparison between conflicts number and peace agreements number

In [ ]:
dt17 = pd.read_csv('2017.csv', parse_dates=['event_date']).set_index('event_date')
dt18 = pd.read_csv('2018.csv', parse_dates=['event_date']).set_index('event_date')
dt19 = pd.read_csv('2019.csv', parse_dates=['event_date']).set_index('event_date')
dt_all = pd.concat([dt17, dt18, dt19])
dt_all['Month'] = pd.Series(dt_all.index).dt.to_period('M').values
dt_all = dt_all.groupby('Month')[['region']].count()
dt_natres = cflct_df.groupby('Month')[['event_id_cnty']].count()
dt_compar= dt_all.join(dt_natres, how = 'inner')
dt_compar.columns = ['all_confilcts',  'peace agreement']
dt_compar.plot(figsize = (12, 8))
plt.title('Comparison between conflicts number and peace agreements number')
plt.ylabel('Numbers')

We cannot exactly identify NatRes/LaEn content in ACLED database. Clearly, there is much more records in this data set, which suggests that we cannot establish one to one linkage between the two data sets. Nevertheless, we try to find the relationship between NatRes/LaEn agreement trend and Conflict trend. Therefore, we are going to find and test if there is some correlations between the NatRes/LaEN and the conflicts numbers or some type of conflict number.


Probably, conflicts for natural resource are linked with war. Therefore, firstly, we try to focus on 'Battle' without geographical limitation. In the next step, we are going to test following 4 hypotheses came up as we explored the data:

- ResNat/LaEn number correlates with Battle event number.
- ResNat/LaEn number correlates with Armed clash event number.
- ResNat/LaEn number correlates with Government regains territory event number.
- ResNat/LaEn number correlates with Non-state actor overtakes territory event number.

battle_df = pd.read_csv('battle.csv',parse_dates=['event_date'])
battle_df.columns

In [ ]:
battle_df[['event_date', 'country','sub_event_type']].describe()

In [ ]:
battle_df[['event_date', 'country','sub_event_type']]

In [ ]:
name, counts = np.unique(battle_df.country, return_counts=True)
cty_df = pd.DataFrame([name, counts], index=['countries', 'counts']).T
most_agreements_cty = cty_df.sort_values('counts', ascending= False).head(10)
most_agreements_cty.set_index('countries').plot(kind = 'bar',figsize = (16,8),rot =35)
plt.title('Top 10 Regions with Natural Resource/Enviroment Relevant Agreement')
plt.ylabel("Number")
plt.show()

In [ ]:
battle_df['Month'] = pd.Series(battle_df.event_date).dt.to_period('M').values


p1 = battle_df.groupby('Month')[['event_id_cnty']].count().pct_change()
p1.columns =['change']

p2 = battle_df.groupby('Month')[['event_id_cnty']].count()
p2.columns =['counts']


In [ ]:
battle_month = pd.concat([p1, p2],axis=1).copy()

In [ ]:
nat_df = pec_df[((pec_df.NatRes==1) | (pec_df.LaEn == 1))]
nat_df['Month'] = pd.Series(nat_df.index).dt.to_period('M').values
p1 = nat_df.groupby('Month')[['Con']].count().pct_change()
p1.columns =['Nat_change']
p2 = nat_df.groupby('Month')[['Con']].count()
p2.columns =['Nat_counts']
nat_month = pd.concat([p1, p2],axis=1).copy()
nat_month.head()

In [ ]:
battle_month.head()

In [ ]:
nat_battle = nat_month.join(battle_month.shift(11)).dropna()
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
trs = scaler.fit_transform(nat_battle)
plt_df = pd.DataFrame(trs,index = nat_battle.index, columns=nat_battle.columns)
plt_df.plot(figsize = (18, 8))
plt.title('Comparison Time Series')
plt.ylabel("Number")
plt.show()

From the figure above, we can not find out clear co-movement between NatRes/LaEn and Battle, and the correlation of them also suggests very weak.

In [ ]:
plt_df.corr()

In [ ]:
types = list(set(battle_df.sub_event_type))
@interact(typeo =types)
def private_public(typeo = types[0]):
    tmp_df =  battle_df.loc[battle_df.sub_event_type == typeo]
    p1 = tmp_df.groupby('Month')[['event_id_cnty']].count().pct_change()
    p1.columns =['sub event change']
    p2 = battle_df.groupby('Month')[['event_id_cnty']].count()
    p2.columns =['sub event counts']
    tmp_month = pd.concat([p1, p2],axis=1).copy()

    nat_df = pec_df[((pec_df.NatRes==1) | (pec_df.LaEn == 1))]
    nat_df['Month'] = pd.Series(nat_df.index).dt.to_period('M').values
    p1 = nat_df.groupby('Month')[['Con']].count().pct_change()
    p1.columns =['Nat_change']
    p2 = nat_df.groupby('Month')[['Con']].count()
    p2.columns =['Nat_counts']
    nat_month = pd.concat([p1, p2],axis=1).copy()
    nat_battle = nat_month.join(tmp_month).dropna()

    scaler = StandardScaler()
    trs = scaler.fit_transform(nat_battle)
    plt_df = pd.DataFrame(trs,index = nat_battle.index, columns=nat_battle.columns)
    df = plt_df.copy()
    x = pd.to_datetime([str(i) for i in df.index])
#     ts = [pd.datetime(x,y,1) for x, y in zip(list(x.month),list(x.month))]
    ts = pd.Series(pd.to_datetime([str(i) for i in plt_df.index]))
    ts = [pd.to_datetime(i) for i in ts]

    xs = [ ts  for i in df.columns] 
    ys = df.T.values.tolist()
    data = {'xs': xs,
        'ys': ys,
        'labels': list(df.columns),
           'color': ['black','blue', 'red','yellow']}

    source = ColumnDataSource(data)  
    p = figure(title='Time Seires Sub Event and NatRes/LaEn')
    p.multi_line(xs='xs', ys='ys', legend='labels',color = 'color', source=source)
    p.xaxis.axis_label = 'time'

    p.legend.location = "top_left"
    p.xaxis.major_label_text_font_size = '0pt'

    show(p)
    print('Correlation Matrix')
    display(df.corr())
    

As shown above, we can not find trends correlation between NatRs/LatEn and Sub events. Therefore, our results are against our hypotheses. 
This is because:
- The correlations in absoulute number counts are low.
- The correlations in realtive percentage changes are also low.
And we have already tested the result using some advanced algorithms and checking each hypotheses individually.

# Reflect and Hypothesis
Reflect: From the whole data analysis process, we first describe our data, including data type, size, fields and summary. And then we carry out a deeper exploration of the data to look for the distribution of the development trends of natural resources/environment peace agreements' number and peace agreements' number over time, which are multimodal distributions, and the relationship between the NatRes/LaEN and the conflicts numbers, or some type of conflict number which further consists of our four hypotheses: ResNat/LaEn number correlates with Battle event number; Armed clash event number; Government regains territory event number; and Non-state actor overtakes territory event number. Finally, we test our hypotheses with advanced algorithms and reach the conclusion that there is very weak correlation. Therefore, it seems that in our real world, the assignment of peace agreement with content of natural resources and environment has weak correlation with those data on conflicts or violent events across the world. This may because there are some more important factors to be assigned to ceasefire.

Hypothesis: ResNat/LaEn number correlates with Battle event number. ResNat/LaEn number correlates with Armed clash event number. ResNat/LaEn number correlates with Government regains territory event number. ResNat/LaEn number correlates with Non-state actor overtakes territory event number. The reason for making these hypotheses is according to the event_type and sub_event_type of battle in ACLED dataset. We try to link peace agreement data with natural resources and environment alongside data of battles in violence events. The test of our hypotheses is accomplished by advanced algorithms above. We reach the conclusion finally.